---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [30]:
import pandas as pd
import numpy as np

def blight_model():
    
    def date_features(df):
        df['ticket_issued_day'] = pd.to_datetime(df['ticket_issued_date']).dt.day
        df['ticket_issued_month'] = pd.to_datetime(df['ticket_issued_date']).dt.month
        df['ticket_issued_year'] = pd.to_datetime(df['ticket_issued_date']).dt.year
        df['ticket_issued_hour'] = pd.to_datetime(df['ticket_issued_date']).dt.hour
        df['ticket_issued_minute'] = pd.to_datetime(df['ticket_issued_date']).dt.minute
        df['hearing_day'] = pd.to_datetime(df['hearing_date']).dt.day
        df['hearing_month'] = pd.to_datetime(df['hearing_date']).dt.month
        df['hearing_year'] = pd.to_datetime(df['hearing_date']).dt.year
        df['hearing_hour'] = pd.to_datetime(df['hearing_date']).dt.hour
        df['hearing_minute'] = pd.to_datetime(df['hearing_date']).dt.minute
        return df
    
    train_df = pd.read_csv('train.csv', encoding='cp1252',dtype={"zip_code":object,"non_us_str_code":object})
    test_df = pd.read_csv('readonly/test.csv',encoding='cp1252',dtype={"zip_code":object,"non_us_str_code":object})
    train_df.drop(["payment_amount","payment_date","payment_status","collection_status","compliance_detail","balance_due"],axis=1,inplace=True)
    train_df['grafitti_status'] = train_df['grafitti_status'].where(train_df['grafitti_status'].isnull(),1).fillna(0).astype(int)
    test_df['grafitti_status'] = test_df['grafitti_status'].where(test_df['grafitti_status'].isnull(),1).fillna(0).astype(int)
    
    #map unique agency_names to an integer value
    agency_map = {k: v for v,k in enumerate(train_df['agency_name'].unique())}
    train_df['agency_name'] = train_df['agency_name'].map(agency_map)
    test_df['agency_name'] = test_df['agency_name'].map(agency_map)

    #map unique inspector_name to an integer value
    inspector_map = {k: v for v,k in enumerate(train_df['inspector_name'].unique())}
    train_df['inspector_name'] = train_df['inspector_name'].map(inspector_map)
    test_df['inspector_name'] = test_df['inspector_name'].map(inspector_map)
    
    #map disposition to an integer value
    disposition_map = {k: v for v,k in enumerate(test_df['disposition'].unique())}
    train_df['disposition'] = train_df['disposition'].map(disposition_map)
    test_df['disposition'] = test_df['disposition'].map(disposition_map)
    
    #map ticket id to lat/lon to use instead of text address, use 0 for misspelled addresses
    addys = pd.read_csv('readonly/addresses.csv')
    lats = pd.read_csv('readonly/latlons.csv')
    addy_lats = pd.merge(addys,lats,on='address')
    addy_lats = addy_lats.fillna(0).drop('address',axis=1)
    train_df = pd.merge(train_df,addy_lats,on='ticket_id')
    test_df = pd.merge(test_df,addy_lats,on='ticket_id')
    
    #map violation descriptions to an integer value. assumes training df contains all possibilities
    violation_map = {k: v for v,k in enumerate(train_df['violation_description'].unique())}
    train_df['violation_description'] = train_df['violation_description'].map(violation_map)
    test_df['violation_description'] = test_df['violation_description'].map(violation_map)
    
    #map violation codes to integer values
    viocode_map = {k: v for v,k in enumerate(train_df['violation_code'].unique())}
    train_df['violation_code'] = train_df['violation_code'].map(viocode_map)
    test_df['violation_code'] = test_df['violation_code'].map(viocode_map)
    
    #map detroit vs not detroit
    is_detroit = {'DETROIT':1,'DET':1}
    train_df['city'] = train_df['city'].str.upper().map(is_detroit).fillna(0)
    test_df['city'] = test_df['city'].str.upper().map(is_detroit).fillna(0)
    
    #map MI vs not MI
    train_df['state'] = train_df['state'].map({'MI':1}).fillna(0)
    test_df['state'] = test_df['state'].map({'MI':1}).fillna(0)
    
    #map USA vs not USA
    train_df['country'] = train_df['country'].map({'USA':1}).fillna(0)
    test_df['country'] = test_df['country'].map({'USA':1}).fillna(0)
    
    #convert date entries to useable features
    train_df = date_features(train_df)
    test_df = date_features(test_df)
    
    #final cleanup of train and test dataframes
    pointless_columns = ['late_fee','violator_name','violation_street_number','violation_street_name','violation_zip_code',
                         'mailing_address_str_number','mailing_address_str_name','zip_code','non_us_str_code']
    train_df = train_df.drop(pointless_columns,axis=1)
    test_df = test_df.drop(pointless_columns,axis=1)
    
    train_df = train_df.dropna(subset=['compliance'])
    X = train_df.drop(['compliance','ticket_issued_date','hearing_date'],axis=1).fillna(0)
    y = train_df['compliance']    
    test_X = test_df.drop(['ticket_issued_date','hearing_date'],axis=1).fillna(0)
        
    from sklearn.ensemble import GradientBoostingClassifier
    
    clf = GradientBoostingClassifier(learning_rate=0.2,random_state=0).fit(X,y)
    y_proba_clf = clf.predict_proba(test_X)
    y_proba_series = pd.Series(y_proba_clf[:,1],index=test_df['ticket_id'])
    #y_proba_list = list(zip(test_df['ticket_id'],y_proba_clf[:,1]))
         
    def score_and_plot():
        
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
        clf = GradientBoostingClassifier(learning_rate = 0.2,random_state=0).fit(X_train,y_train)
        y_proba_clf = clf.predict_proba(X_test)
        y_proba_list = list(zip(y_test[0:30],y_proba_clf[0:30,1]))
        
        print('Gradient Boosting, Blight dataset, learning_rate = 0.2')
        print('Accuracy of GB classifier on training set: {:.2f}'
         .format(clf.score(X_train, y_train)))
        print('Accuracy of GB classifier on test set: {:.2f}'
         .format(clf.score(X_test, y_test)))
        
        from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
        predicted = clf.predict(X_test)
        print('Accuracy: {:.2f}'.format(accuracy_score(y_test,predicted)))
        print('Precision: {:.2f}'.format(precision_score(y_test,predicted)))
        print('Recall: {:.2f}'.format(recall_score(y_test,predicted)))
        print('F1: {:.2f}'.format(f1_score(y_test,predicted)))

        y_score_clf = clf.decision_function(X_test)
        fpr,tpr, _ = roc_curve(y_test, y_score_clf)
        roc_auc_clf = auc(fpr,tpr)

        %matplotlib notebook
        import matplotlib.pyplot as plt


        plt.figure()
        plt.xlim([-0.01, 1.00])
        plt.ylim([-0.01, 1.01])
        plt.plot(fpr, tpr, lw=3, label='GBC ROC curve (area = {:0.2f})'.format(roc_auc_clf))
        plt.xlabel('False Positive Rate', fontsize=16)
        plt.ylabel('True Positive Rate', fontsize=16)
        plt.title('ROC curve (Gradient Boosting Classifier)', fontsize=16)
        plt.legend(loc='lower right', fontsize=13)
        plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
        plt.axes().set_aspect('equal')
        plt.show()
    #score_and_plot() #reports metrics for training data Accuracy is 0.94, ROCAUC is 0.83, Precision is 0.90
    
    return y_proba_series

In [31]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.090199
285362    0.014541
285361    0.095874
285338    0.088467
285346    0.140577
285345    0.088467
285347    0.134055
285342    0.733829
285530    0.030066
284989    0.041795
285344    0.162986
285343    0.036394
285340    0.022018
285341    0.114522
285349    0.140577
285348    0.088467
284991    0.041795
285532    0.026356
285406    0.025582
285001    0.109692
285006    0.046287
285405    0.026056
285337    0.032293
285496    0.138362
285497    0.104551
285378    0.025774
285589    0.033811
285585    0.059193
285501    0.144316
285581    0.014381
            ...   
376367    0.369032
376366    0.523930
376362    0.494244
376363    0.577423
376365    0.369032
376364    0.523930
376228    0.161558
376265    0.575990
376286    0.853471
376320    0.102335
376314    0.065658
376327    0.478926
376385    0.671960
376435    0.487665
376370    0.922497
376434    0.206142
376459    0.168047
376478    0.031721
376473    0.104082
376484    0.086916
376482    0.076685
37